In [4]:
%load_ext autoreload
%autoreload 2

import torch
from training import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
NUM_CLIENTS = 2
ACCURACY_THRESHOLD = 93

In [6]:
#Load data
train_loaders, val_loader, test_loader = get_data_loaders(BATCH_SIZE, NUM_CLIENTS)
print(len(train_loaders))

#Initialize all clients
clients = []
for train_loader in train_loaders:
    client = Client(train_loader)
    clients.append(client)
torch.manual_seed(clients[0].seed)

* Using MNIST
** Reduce the data-set (use --full for the full thing)
** Use 5000 train and 5000 test samples
2


In [7]:
testing_accuracy = 0
bits_transferred = 0
num_rounds = 0
bits_conserved = 0

centralServer = Client(test_loader)

while testing_accuracy < ACCURACY_THRESHOLD:
    num_rounds += 1
    print("Communication Round {0}".format(num_rounds))
    
    if num_rounds > 1:
        #Load server weights onto clients
        #TODO: Count number of transferred bits
        for client in clients:
            with torch.no_grad():
                client.model.load_state_dict(dict(centralServer.model.named_parameters()))
                
    #Perform E local training steps for each client
    for client_idx, client in enumerate(clients):
        print("Training client {0}".format(client_idx))
        for epoch in range(1, client.epochs + 1):
            train(client, epoch)

    #Average model parameters and test
    #TODO: Count number of transfered bits and sparsify gradients
    with torch.no_grad():
        dict_params = averageClientModels(clients)
        centralServer.model.load_state_dict(dict_params)
    test_loss, testing_accuracy = test(centralServer)

#Save model
if centralServer.save_model:
     torch.save(centralServer.model.state_dict(), f"{centralServer.model_name}.pt")

Communication Round 1
Training client 0
Train Epoch: 1 Loss: 0.523767, Train accuracy: 87.55
Train Epoch: 2 Loss: 0.211090, Train accuracy: 92.5
Train Epoch: 3 Loss: 0.146634, Train accuracy: 94.75
Train Epoch: 4 Loss: 0.259047, Train accuracy: 96.85
Train Epoch: 5 Loss: 0.082258, Train accuracy: 98.0
Training client 1
Train Epoch: 1 Loss: 0.545741, Train accuracy: 84.35
Train Epoch: 2 Loss: 0.445594, Train accuracy: 91.35
Train Epoch: 3 Loss: 0.353354, Train accuracy: 95.15
Train Epoch: 4 Loss: 0.109376, Train accuracy: 96.35
Train Epoch: 5 Loss: 0.153142, Train accuracy: 98.15
Test set: Average loss: 0.0398, Test accuracy: 1479 / 5000 (30%)

Communication Round 2
Training client 0
Train Epoch: 1 Loss: 0.464770, Train accuracy: 88.95
Train Epoch: 2 Loss: 0.211386, Train accuracy: 92.55
Train Epoch: 3 Loss: 0.169998, Train accuracy: 95.1
Train Epoch: 4 Loss: 0.094199, Train accuracy: 96.0
Train Epoch: 5 Loss: 0.233825, Train accuracy: 95.7
Training client 1
Train Epoch: 1 Loss: 0.43458